In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [12]:
df = pd.read_csv('Google_Stock_Price_Train.csv')
training_set = df.iloc[:,1:2].values
df.shape

(1258, 6)

In [13]:
# feature scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
training_set_scaled = sc.fit_transform(training_set)

In [14]:
# creating data with 60 timesteps
X_train = []
y_train = []
# till 59 th time step we look at the stock open values that is our training data
# our value at the 60 th step ie we are trying to predict is our y_train
for i in range(60,1258):
    X_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])
X_train,y_train = np.array(X_train),np.array(y_train)
# reshaping
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))

In [16]:
import tensorflow as tf

keras.engine.sequential.Sequential

In [20]:
#initialize the RNN
RNN = tf.keras.models.Sequential()

### Adding LSTM layers with some Dropout regularization

In [18]:
# Adding the LSTM layers and some dropout regularization

